### Importando bibliotecas necessárias

In [38]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import requests

### Baixar arquivos:

In [39]:
"""
url = 'https://files.grouplens.org/datasets/movielens/ml-1m.zip'

try:
    response = requests.get(url, verify=False)
    with open('ml-1m.zip', 'wb') as file:
        file.write(response.content)
    print("Download concluído com sucesso!")
except requests.exceptions.RequestException as e:
    print(f"Erro ao baixar o arquivo: {e}")

"""

'\nurl = \'https://files.grouplens.org/datasets/movielens/ml-1m.zip\'\n\ntry:\n    response = requests.get(url, verify=False)\n    with open(\'ml-1m.zip\', \'wb\') as file:\n        file.write(response.content)\n    print("Download concluído com sucesso!")\nexcept requests.exceptions.RequestException as e:\n    print(f"Erro ao baixar o arquivo: {e}")\n\n'

In [40]:
"""
url = 'https://files.grouplens.org/datasets/movielens/ml-1m-README.txt'

try:
    response = requests.get(url, verify=False)
    with open('ml-1m.zip', 'wb') as file:
        file.write(response.content)
    print("Download concluído com sucesso!")
except requests.exceptions.RequestException as e:
    print(f"Erro ao baixar o arquivo: {e}")
"""

'\nurl = \'https://files.grouplens.org/datasets/movielens/ml-1m-README.txt\'\n\ntry:\n    response = requests.get(url, verify=False)\n    with open(\'ml-1m.zip\', \'wb\') as file:\n        file.write(response.content)\n    print("Download concluído com sucesso!")\nexcept requests.exceptions.RequestException as e:\n    print(f"Erro ao baixar o arquivo: {e}")\n'

In [41]:
# Lendo o arquivo movies.dat
df_movies = pd.read_csv(
    r"C:\Users\thiag\OneDrive\Desktop\datamining\movie_cluster\data\ml-1m\movies.dat",
    delimiter='::',
    engine='python',
    names=['MovieID', 'Title', 'Genres'],
    encoding='latin1'
)

# Lendo o arquivo ratings.dat
df_ratings = pd.read_csv(
    r"C:\Users\thiag\OneDrive\Desktop\datamining\movie_cluster\data\ml-1m\ratings.dat",
    delimiter='::',
    engine='python',
    names=['UserID', 'MovieID', 'Rating', 'Timestamp'],
    encoding='latin1'
)

# Lendo o arquivo users.dat
df_users = pd.read_csv(
    r"C:\Users\thiag\OneDrive\Desktop\datamining\movie_cluster\data\ml-1m\users.dat",
    delimiter='::',
    engine='python',
    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'],
    encoding='latin1'
)

print(df_movies.head())
print(df_ratings.head())
print(df_users.head())


   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291
   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455
